In [46]:
import random
import threading
import time
import numpy as np
from ipywidgets import IntProgress
from IPython.display import display

In [47]:

class Generation_data:
    def __init__(self, nb_objets, nbres_camions):
        self.nb_objets = nb_objets
        self.nbres_camions = nbres_camions
        self.grand_tableau = self.generer_tableau()
        self.tableau_camion = self.gen_tab_camion()

    def generer_tableau(self):
        nb_villes = self.nb_objets * 2
        tab_ville = list(range(1, nb_villes + 1))
        grand_tableau = []

        for _ in range(nb_villes // 2):
            ville_collecte = tab_ville.pop(random.randint(0, len(tab_ville) - 1))
            ville_livraison = tab_ville.pop(random.randint(0, len(tab_ville) - 1))
            camion = random.randint(1, self.nbres_camions)
            grand_tableau.append([ville_collecte, ville_livraison, [camion]])

        return grand_tableau

    def gen_tab_camion(self):
        tab_camion = [[] for _ in range(self.nbres_camions)]

        for route in self.grand_tableau:
            camion_index = route[2][0] - 1
            tab_camion[camion_index].extend([route[0], route[1]])

        return tab_camion

    def affichage_camions(self):
        for i, camions in enumerate(self.tableau_camion):
            print(f"Camion {i + 1} : {camions}")


    def affichage_1_camion(self, id):
        return self.tableau_camion[id - 1]
        
def random_temps_trajet(temps_min, temps_max, nb_villes):
    temps_trajet = []
    for i in range(nb_villes):
        ligne = []
        for j in range(nb_villes):
            if i == j:
                ligne.append(0)
            elif i < j:
                temps = random.randint(temps_min, temps_max)
                ligne.append(temps)
            else:
                ligne.append(temps_trajet[j][i])
        temps_trajet.append(ligne)
    return temps_trajet

In [48]:
def pheromones_update(pheromone_value, alpha, point_0):
    return pheromone_value * (1 - alpha) + alpha * point_0

def calculate_probabilities(current_index, points, pheromone, distance_matrix, alpha, beta, indextab):
    probabilities = []
    total = 0
    for i in range(len(points)):
        if i in indextab:
            probabilities.append(0)
        else:
            pheromone_level = pheromone[current_index][i] ** alpha
            visibility = (1 / distance_matrix[current_index][i]) ** beta if distance_matrix[current_index][i] > 0 else 0
            probability = pheromone_level * visibility
            total += probability
            probabilities.append(probability)
    
    if total == 0:
        return [1 / len(points)] * len(points)  # Si toutes les probabilités sont nulles, retourne des probabilités uniformes
    
    probabilities = [p / total for p in probabilities]
    return probabilities

def choose_next_index(probabilities):
    return np.random.choice(len(probabilities), p=probabilities)

def change_nodes(indextab, poidpath, distance_matrix, points, evaporation_rate, current_nodes, pheromone, alpha, beta):
    point_0 = current_nodes
    
    # Trouver l'index du point courant
    index = points.index(point_0)
    
    # Ajouter l'index courant à l'historique des index
    indextab.append(index)
    
    # Calculer les probabilités pour choisir le prochain index
    probabilities = calculate_probabilities(index, points, pheromone, distance_matrix, alpha, beta, indextab)
    
    # Choisir un nouvel index basé sur les probabilités
    newindex = choose_next_index(probabilities)
    
    # Mise à jour du poids du chemin
    if len(indextab) > 1:
        previousindex = indextab[-2]
    else:
        previousindex = index  # initialisez previousindex à index si c'est la première itération
    
    poidpath += distance_matrix[previousindex][index]
    #print(f'Poidpath après ajout: {poidpath}')
    
    # Mettre à jour l'index courant avec le nouvel index
    index = newindex
    point_0 = points[index]
    
    # Mettre à jour les phéromones
    pheromone[previousindex][index] = pheromones_update(pheromone[previousindex][index], alpha, 1.0)
    
    return point_0, poidpath

def ant_colony_optimization(k, points, n_ants, n_iterations, alpha, beta, evaporation_rate):
    best_path = []
    best_path_length = float('inf')
    ##distance_matrix = random_temps_trajet(40, 100, len(points))
    pheromone = initialize_pheromones(len(points))
    bar1 = IntProgress(min=0, max=n_iterations, layout={"width" : "100%"})
    display(bar1)
    for iteration in range(n_iterations):
        bar1.value += 1
        for ant in range(n_ants):
            path = []
            indextab = []
            poidpath = 0
            current_point = points[0]
            
            while len(path) < len(points):
                path.append(current_point)
                current_point, poidpath = change_nodes(indextab, poidpath, distance_matrix, points, evaporation_rate, current_point, pheromone, alpha, beta)
            path.append(points[0])
            poidpath += distance_matrix[points.index(current_point)][0]
            if poidpath < best_path_length:
                best_path_length = poidpath
                best_path = path
            
        pheromone = evaporate_pheromones(pheromone, evaporation_rate)
    print(f'Truck : ', k)
    print(f'Best path length: {best_path_length}')
    print(f'Best path: {best_path}')
    bar1.close()
    return best_path_length
    
def initialize_pheromones(num_nodes):  
    return np.ones((num_nodes, num_nodes))

def evaporate_pheromones(pheromone, evaporation_rate):
    return pheromone * (1 - evaporation_rate)



In [49]:

def mono_thread(k):
    camion_list = logistics.affichage_1_camion(k)
    camion_list.insert(0, 0)
    print("camion ", k, " : ", camion_list)
    
    var_sortie_fonction_traitement=ant_colony_optimization(k, camion_list, n_ants, n_iterations, alpha, beta, evaporation_rate)

    
    with results_lock:
        results.append(var_sortie_fonction_traitement)
    
def launch_threads(k):
    threads = []

    start_time = time.time()  # Start 

    for i in range(k):
        thread = threading.Thread(target=mono_thread, args=(i + 1,))
        threads.append(thread)
        thread.start()

    for thread in threads:
        thread.join()
    
    end_time = time.time()  # Stop
    
    total_duration = end_time - start_time
    print("")
    print(f"Longest duration for all threads: {total_duration:.4f} seconds")
    print("End of all the thread")
    
    print("Results: ", max(results))


In [51]:
results = []
results_lock = threading.Lock()
random.seed(a=3)

nombres_objets=50
nombres_villes = nombres_objets * 2

nombres_camions = 5
n_ants = 10
n_iterations = 100
alpha = 1
beta = 3
evaporation_rate = 0.8

distance_matrix = random_temps_trajet(40, 100, 2*nombres_objets+1)

logistics = Generation_data(nombres_objets, nombres_camions)
#logistics.affichage_camions()
launch_threads(nombres_camions)

camion  1  :  [0, 43, 74, 36, 63, 39, 14, 97, 82, 16, 29, 7, 80, 53, 99, 59, 62, 79, 9, 10, 56, 83, 18]
camion  2  :  [0, 17, 75, 42, 89, 4, 76, 96, 11, 6, 5, 26, 25, 13, 21, 98, 2, 64, 37]
camion  3  :  [0, 24, 12, 86, 84, 70, 61, 20, 30, 15, 52, 22, 65, 66, 28, 92, 57]
camion  4  :  [0, 91, 19, 48, 68, 33, 27, 44, 95, 49, 58, 23, 8, 51, 1]
camion  5  :  [0, 35, 71, 72, 3, 45, 73, 31, 60, 55, 85, 40, 47, 32, 94, 88, 100, 46, 67, 41, 54, 90, 50, 77, 87, 38, 78, 93, 34, 69, 81]


IntProgress(value=0, layout=Layout(width='100%'))

IntProgress(value=0, layout=Layout(width='100%'))

IntProgress(value=0, layout=Layout(width='100%'))

IntProgress(value=0, layout=Layout(width='100%'))

IntProgress(value=0, layout=Layout(width='100%'))

Truck :  4
Best path length: 755
Best path: [0, 51, 48, 1, 95, 19, 8, 58, 27, 68, 33, 23, 49, 91, 44, 0]
Truck :  3
Best path length: 826
Best path: [0, 66, 57, 20, 70, 84, 61, 52, 65, 28, 30, 15, 12, 92, 24, 86, 22, 0]
Truck :  2
Best path length: 913
Best path: [0, 17, 5, 75, 98, 96, 76, 26, 21, 42, 6, 37, 4, 89, 25, 2, 13, 64, 11, 0]
Truck :  1
Best path length: 1097
Best path: [0, 53, 62, 99, 36, 43, 59, 29, 63, 9, 14, 74, 83, 79, 80, 56, 97, 10, 39, 82, 18, 7, 16, 0]
Truck :  5
Best path length: 1572
Best path: [0, 40, 81, 71, 78, 85, 47, 87, 94, 60, 50, 69, 45, 55, 3, 38, 72, 100, 31, 73, 46, 90, 32, 67, 93, 54, 41, 88, 35, 77, 34, 0]

Longest duration for all threads: 5.5985 seconds
End of all the thread
Results:  1572
